# SQL tutorial using MySQL and python connector

In [1]:
#pip install mysql-connector-python

In [ ]:
from os import environ
from mysql.connector import connect, Error

In [2]:
##Alternative way to ask for the password without showing it while typing

#from getpass import getpass

#user=input("Enter username: ")
#password=getpass("Enter password: ")

In [3]:
## Import credentials from environment variables
user=environ.get('MySQL_User')
password=environ.get('MySQL_psswrd')

## Establish connection with the server

In [4]:
try:
    with connect(host="localhost",user=user,password=password) as connection:
        print(connection)
except Error as e:
    print("Error: ",e)

## Create a new database

In [5]:
create_db_query = "CREATE DATABASE online_movie_rating"

try:
    with connect(host="localhost",user=user,password=password) as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print("Error: ",e)

Error:  1007 (HY000): Can't create database 'online_movie_rating'; database exists


## Show existing databases

In [6]:
show_db_query = "SHOW DATABASES"

try:
    with connect(host="localhost",user=user,password=password) as connection:
        with connection.cursor() as cursor:
            cursor.execute(show_db_query)
            for db in cursor:
                print(db)
except Error as e:
    print("Error: ",e)

('information_schema',)
('mysql',)
('online_movie_rating',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


## Connecting to an already existing database

In [7]:
try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        print(connection)
except Error as e:
    print("Error: ",e)

## Creating new tables in the database

In [8]:
#Define the create table queries

create_genres_table_query = """
CREATE TABLE genres (
    id INT AUTO_INCREMENT PRIMARY KEY,
    genre VARCHAR(100) NOT NULL,
    UNIQUE(genre)
)
"""

create_movies_table_query = """
CREATE TABLE movies(
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    release_year YEAR(4) NOT NULL,
    genre_id INT,
    collection_in_mil INT,
    FOREIGN KEY (genre_id) REFERENCES genres(id),
    CONSTRAINT UC_movie UNIQUE(title,release_year)
)
"""

create_reviewers_table_query = """
CREATE TABLE reviewers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100)
    CONSTRAINT UC_movie UNIQUE(first_name,last_name)
)
"""

#CONSTRAINT UC_reviewer UNIQUE(first_name,last_name)

create_ratings_table_query = """
CREATE TABLE ratings (
    movie_id INT,
    reviewer_id INT,
    rating DECIMAL(2,1) NOT NULL,
    FOREIGN KEY(movie_id) REFERENCES movies(id),
    FOREIGN KEY(reviewer_id) REFERENCES reviewers(id),
    PRIMARY KEY(movie_id, reviewer_id),
    CHECK (rating<=10.0)
)
"""

In [9]:
try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_genres_table_query)
            print("ok")
            cursor.execute(create_movies_table_query)
            print("ok")
            cursor.execute(create_reviewers_table_query)
            print("ok")
            cursor.execute(create_ratings_table_query)
            print("ok")
            connection.commit()
except Error as e:
    print("Error: ",e)

ok
ok
ok
ok


## Show table scheme

In [10]:
show_table_query = "DESCRIBE movies"

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(show_table_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'NO', 'MUL', None, '')
('release_year', b'year', 'NO', '', None, '')
('genre_id', b'int', 'YES', 'MUL', None, '')
('collection_in_mil', b'int', 'YES', '', None, '')


## Editing tables scheme

In [11]:
alter_table_query = """
ALTER TABLE movies
MODIFY COLUMN collection_in_mil DECIMAL(4,1)
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(alter_table_query)
            cursor.execute(show_table_query)
            # Fetch rows from last executed query
            result = cursor.fetchall()
            print("Movie Table Schema after alteration:")
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

Movie Table Schema after alteration:
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'NO', 'MUL', None, '')
('release_year', b'year', 'NO', '', None, '')
('genre_id', b'int', 'YES', 'MUL', None, '')
('collection_in_mil', b'decimal(4,1)', 'YES', '', None, '')


## Deleting tables

In [12]:
drop_table_query = "DROP TABLE ratings"
#drop_table_query = "DROP TABLE genres"
#drop_table_query = "DROP TABLE movies"
#drop_table_query = "DROP TABLE reviewers"

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(drop_table_query)
except Error as e:
    print("Error: ",e)

In [13]:
try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_ratings_table_query)
            connection.commit()
except Error as e:
    print("Error: ",e)

## Inserting records

In [14]:
insert_genres_query = """
INSERT INTO genres (genre)
VALUES
    ("Drama"),
    ("Action"),
    ("Romance"),
    ("Comedy"),
    ("Horror"),
    ("Crime"),
    ("Adventure"),
    ("Animation"),
    ("Sport"),
    ("War"),
    ("Thriller"),
    ("Mistery")
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(insert_genres_query)
            connection.commit()
except Error as e:
    print("Error: ",e)

In [15]:
insert_movies_query = """
INSERT INTO movies (title, release_year, genre_id, collection_in_mil)
VALUES
    ("Forrest Gump", 1994, 1, 330.2),
    ("3 Idiots", 2009, 1, 2.4),
    ("Eternal Sunshine of the Spotless Mind", 2004, 1, 34.5),
    ("Good Will Hunting", 1997, 1, 138.1),
    ("Skyfall", 2012, 2, 304.6),
    ("Gladiator", 2000, 2, 188.7),
    ("Black", 2005, 1, 3.0),
    ("Titanic", 1997, 3, 659.2),
    ("The Shawshank Redemption", 1994, 1,28.4),
    ("Udaan", 2010, 1, 1.5),
    ("Home Alone", 1990, 4, 286.9),
    ("Casablanca", 1942, 3, 1.0),
    ("Avengers: Endgame", 2019, 2, 858.8),
    ("Night of the Living Dead", 1968, 5, 2.5),
    ("The Godfather", 1972, 6, 135.6),
    ("Haider", 2014, 2, 4.2),
    ("Inception", 2010, 7, 293.7),
    ("Evil", 2003, 5, 1.3),
    ("Toy Story 4", 2019, 8, 434.9),
    ("Air Force One", 1997, 1, 138.1),
    ("The Dark Knight", 2008, 2,535.4),
    ("Bhaag Milkha Bhaag", 2013, 9, 4.1),
    ("The Lion King", 1994, 8, 423.6),
    ("Pulp Fiction", 1994, 6, 108.8),
    ("Kai Po Che", 2013, 9, 6.0),
    ("Beasts of No Nation", 2015, 10, 1.4),
    ("Andadhun", 2018, 11, 2.9),
    ("The Silence of the Lambs", 1991, 6, 68.2),
    ("Deadpool", 2016, 2, 363.6),
    ("Drishyam", 2015, 12, 3.0)
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(insert_movies_query)
            connection.commit()
except Error as e:
    print("Error: ",e)

In [16]:
###EXECUTEMANY

insert_reviewers_query = """
INSERT INTO reviewers
(first_name, last_name)
VALUES ( %s, %s )
"""
reviewers_records = [
    ("Chaitanya", "Baweja"),
    ("Mary", "Cooper"),
    ("John", "Wayne"),
    ("Thomas", "Stoneman"),
    ("Penny", "Hofstadter"),
    ("Mitchell", "Marsh"),
    ("Wyatt", "Skaggs"),
    ("Andre", "Veiga"),
    ("Sheldon", "Cooper"),
    ("Kimbra", "Masters"),
    ("Kat", "Dennings"),
    ("Bruce", "Wayne"),
    ("Domingo", "Cortes"),
    ("Rajesh", "Koothrappali"),
    ("Ben", "Glocker"),
    ("Mahinder", "Dhoni"),
    ("Akbar", "Khan"),
    ("Howard", "Wolowitz"),
    ("Pinkie", "Petit"),
    ("Gurkaran", "Singh"),
    ("Amy", "Farah Fowler"),
    ("Marlon", "Crafford"),
]

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.executemany(insert_reviewers_query,reviewers_records)
            connection.commit()
except Error as e:
    print("Error: ",e)

In [17]:
insert_ratings_query = """
INSERT INTO ratings
(rating, movie_id, reviewer_id)
VALUES ( %s, %s, %s)
"""

ratings_records = [
    (6.4, 17, 5), (5.6, 19, 1), (6.3, 22, 14), (5.1, 21, 17),
    (5.0, 5, 5), (6.5, 21, 5), (8.5, 30, 13), (9.7, 6, 4),
    (8.5, 24, 12), (9.9, 14, 9), (8.7, 26, 14), (9.9, 6, 10),
    (5.1, 30, 6), (5.4, 18, 16), (6.2, 6, 20), (7.3, 21, 19),
    (8.1, 17, 18), (5.0, 7, 2), (9.8, 23, 3), (8.0, 22, 9),
    (8.5, 11, 13), (5.0, 5, 11), (5.7, 8, 2), (7.6, 25, 19),
    (5.2, 18, 15), (9.7, 13, 3), (5.8, 18, 8), (5.8, 30, 15),
    (8.4, 21, 18), (6.2, 23, 16), (7.0, 10, 18), (9.5, 30, 20),
    (8.9, 3, 19), (6.4, 12, 2), (7.8, 12, 22), (9.9, 15, 13),
    (7.5, 20, 17), (9.0, 25, 6), (8.5, 23, 2), (5.3, 30, 17),
    (6.4, 5, 10), (8.1, 5, 21), (5.7, 22, 1), (6.3, 28, 4),
    (9.8, 13, 1)
]

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.executemany(insert_ratings_query,ratings_records)
            connection.commit()
except Error as e:
    print("Error: ",e)

## Read records and filters

In [18]:
select_movies_query = "SELECT * FROM movies LIMIT 5"

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_movies_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

(1, 'Forrest Gump', 1994, 1, Decimal('330.2'))
(2, '3 Idiots', 2009, 1, Decimal('2.4'))
(3, 'Eternal Sunshine of the Spotless Mind', 2004, 1, Decimal('34.5'))
(4, 'Good Will Hunting', 1997, 1, Decimal('138.1'))
(5, 'Skyfall', 2012, 2, Decimal('304.6'))


In [33]:
select_movies_query = """
SELECT title,collection_in_mil
FROM movies
WHERE collection_in_mil > 300
ORDER BY collection_in_mil DESC
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_movies_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

('Avengers: Endgame', Decimal('858.8'))
('Titanic', Decimal('659.2'))
('The Dark Knight', Decimal('535.4'))
('Toy Story 4', Decimal('434.9'))
('The Lion King', Decimal('423.6'))
('Deadpool', Decimal('363.6'))
('Forrest Gump', Decimal('330.2'))
('Skyfall', Decimal('304.6'))


In [20]:
select_movies_query = """
SELECT CONCAT(title, " (", release_year, ")"),collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
LIMIT 5
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_movies_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))
('Toy Story 4 (2019)', Decimal('434.9'))
('The Lion King (1994)', Decimal('423.6'))


In [21]:
select_movies_query = """
SELECT CONCAT(title, " (", release_year, ")"),collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_movies_query)
            for movie in cursor.fetchmany(size=5):
                print(movie)
            cursor.fetchall()
except Error as e:
    print("Error: ",e)

('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))
('Toy Story 4 (2019)', Decimal('434.9'))
('The Lion King (1994)', Decimal('423.6'))


In [22]:
try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_movies_query)
            print(cursor.fetchone())
            cursor.fetchall()       #needed to clean the cursor, otherwise error is obtained in following action
except Error as e:
    print("Error: ",e)

('Avengers: Endgame (2019)', Decimal('858.8'))


In [23]:
select_movies_query = """
SELECT title, AVG(rating) as average_rating
FROM ratings
INNER JOIN movies
    ON movies.id = ratings.movie_id
GROUP BY movie_id
ORDER BY average_rating DESC
LIMIT 5
"""
    
try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_movies_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

('Night of the Living Dead', Decimal('9.90000'))
('The Godfather', Decimal('9.90000'))
('Avengers: Endgame', Decimal('9.75000'))
('Eternal Sunshine of the Spotless Mind', Decimal('8.90000'))
('Beasts of No Nation', Decimal('8.70000'))


In [24]:
select_reviewers_query = """
SELECT CONCAT(first_name, " ", last_name), COUNT(*) as num
FROM reviewers
INNER JOIN ratings
    ON reviewers.id = ratings.reviewer_id
GROUP BY reviewer_id
ORDER BY num DESC
LIMIT 1
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_reviewers_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

('Mary Cooper', 4)


## Update records

In [25]:
update_query = """
UPDATE
    reviewers
SET
    last_name = "Cooper"
WHERE
    first_name = "Amy"
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(update_query)
            connection.commit()
except Error as e:
    print("Error: ",e)

## Delete records

In [26]:
select_ratings_query = """
SELECT reviewer_id, movie_id FROM ratings
WHERE reviewer_id = 2
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(select_ratings_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

(2, 7)
(2, 8)
(2, 12)
(2, 23)


In [27]:
delete_query = "DELETE FROM ratings WHERE reviewer_id = 2"

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(delete_query)
            connection.commit()
            cursor.execute(select_ratings_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

## EXTRA: Preparing db for new_entry command

In [28]:
delete_ratings_query = "DELETE FROM ratings"
delete_reviewers_query = "DELETE FROM reviewers"
try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(delete_ratings_query)
            cursor.execute(delete_reviewers_query)
            connection.commit()
            cursor.execute(select_ratings_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print("Error: ",e)

In [29]:
alter_table_query = """
ALTER TABLE reviewers
ADD COLUMN user_name VARCHAR(100) NOT NULL,
ADD CONSTRAINT UNIQUE (user_name) 
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(alter_table_query)
            cursor.execute(show_table_query)
            # Fetch rows from last executed query
            result = cursor.fetchall()
except Error as e:
    print("Error: ",e)

In [30]:
insert_reviewers_query = """
INSERT INTO reviewers
(user_name,first_name, last_name)
VALUES ( "pablos-p", "Pablo", "Sánchez-Palencia" )
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(insert_reviewers_query)
            connection.commit()
except Error as e:
    print("Error: ",e)

In [31]:
search_reviewers_query = """
SELECT * FROM reviewers
"""

try:
    with connect(host="localhost",user=user,password=password,database="online_movie_rating") as connection:
        with connection.cursor() as cursor:
            cursor.execute(search_reviewers_query)
            result=cursor.fetchall()
            for reviewer in result:
                print (reviewer)
except Error as e:
    print("Error: ",e)

(23, 'Pablo', 'Sánchez-Palencia', 'pablos-p')
